## Read and Save Query Status in CSV for all Workflows

<br>

**Read all the JSON files for all the workflows and print out the messages and query status to a CSV file**

<br>

In [1]:
# import all the modules. NB: submit_run_ars_modules contains all the modules to submit job to ARAX

import json
import requests
from gamma_viewer import GammaViewer
from IPython.display import display
#from submit_run_ars_modules import submit_to_ars, submit_to_devars, printjson, retrieve_devars_results
import glob 
import os
from collections import defaultdict
import pandas as pd
from time import sleep
from os import path
from datetime import datetime
import numpy as np


In [2]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [3]:
def submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

##https://ars.ci.transltr.io/ars/api

def retrieve_ars_results(mid, name, check_sheet, ars_url='https://ars-dev.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    dict3 = {}
    for child in j['children']:
        print(child['status'])
        error_code = child['code']
        
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                    
                    if name in list(check_sheet.Workflow):
                        print(name)
                        dfy = check_sheet[check_sheet['Workflow']== name]

                        dfy.reset_index(drop=True)

                        for index,curie_id in enumerate(dfy.Curie):
                            print(index,curie_id)
                            node_num = dfy.iloc[index][3]
                            query_id = curie_id
                            if np.isnan(dfy.iloc[index][2]) == True:
                                len_check = len(child_response['fields']['data']['message']['results'])
                            else:
                                len_check = dfy.iloc[index][2]

                            #print(node_num, query_id, len_check)

                            locs = []
                            for x, val in enumerate(child_response['fields']['data']['message']['results']):
                                #print(val)

                                if x < len_check:

                                    if query_id in val['node_bindings'][node_num][0]['id']:
                                        locs.append(x)
                            if not locs:
                                check_result = f'False'
                                print(check_result)
                                #pass
                            else:
                                check_result = f'True'
                                print('curie id:', query_id, ': INCLUDED at postion N ==', locs, 'on', node_num)

                            dict3[curie_id] = check_result    

                if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                    if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                            test_att_values =[]
                            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                                #print(val)
                                
                                for tx in edge_ex[val]['attributes']:
                                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source') :
                                        
                                        
                                        value_att = tx['value']
                        
                                        test_att_values.append(value_att)
                                        test_att = set(flatten_list(test_att_values))
                                        
                                        
                                        dictionary_2[child['actor']['agent']] = test_att
                    #else:
                        #dictionary_2[child['actor']['agent']] = [] 
                #else:
                   # dictionary_2[child['actor']['agent']] = []
            
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
                
            
        
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                #dictionary_2[child['actor']['agent']] = []
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
        
        
        else:
            nresults = 0
            #dictionary_2[child['actor']['agent']] = []
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error' ': ' + str(error_code)
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' ': ' + str(error_code) + ' ' + str(dict3)
        elif ((child['status'] == 'Unknown') & (nresults == 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Unknown' ': ' + str(error_code)
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return [dictionary, dictionary_2]


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))
    
def make_hyperlink(value):
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

<br>

**The below code reads each JSON files from the Workflows A through D (subdirectories). The queries are submitted to ARAX and output is saved in a dictionary, where the key is the file name of the JSON to denote which query is being run and the values assigned to the key is the query id**

<br>


In [4]:
check_sheet = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/Query results to include or exclude - Sheet1.csv", header=0)

In [5]:
check_sheet

,Workflow,Curie,N (size of list of results),Query node ID,Include/Exclude,Unnamed: 5,Unnamed: 6
0,A.0_RHOBTB2_direct.json,PUBCHEM.COMPOUND:2950270,10.0,n1,Include,NaN,Note: this example row means that for workflow...
1,B.1_DILI-three-hop-interacts-with.json,CHEBI:41879,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
2,B.1_DILI-three-hop-interacts-with.json,MESH:D000077185,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
3,B.1_DILI-three-hop-interacts-with.json,MESH:D000077385,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
4,B.1_DILI-three-hop-interacts-with.json,MESH:D003474,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
5,B.1_DILI-three-hop-interacts-with.json,RXNORM:40068,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
6,B.1_DILI-three-hop-interacts-with.json,PUBCHEM.COMPOUND:5877,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
7,B.1_DILI-three-hop-interacts-with.json,PUBCHEM.COMPOUND:5755,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
8,C.1a_SmallMolecule_real_world_evidence_MultScl...,MONDO:0005301,100.0,n00,include,NaN,NaN
9,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallM...,CHEBI:45783,100.0,n3,include,NaN,NaN


In [7]:
PATH = r'/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo'
EXT = "*.json"
dict_workflows = {}
for root, dirs, files in os.walk(PATH): # step 1: accessing file
    for name in files:
        
        if name.endswith((".json")):
            if (name == 'C.1_Template_SmallMolecule_real_world_evidence_Disease.json') or (name == 'C.2_Template_Drug_Disease_GeneSet_interacts_with_SmallMolecule.json') or (name == 'C.3_Template_Disease_related_to_Drug.json'):
                pass
            else:
                file_read = path.join(root, name)
                dir_name = (os.path.splitext(os.path.basename(root))[0])
                print(file_read)

                filename = (os.path.splitext(os.path.basename(file_read))[0])
                print(filename)
                with open(file_read,'r') as inf:
                    query = json.load(inf)

                    kcresult = submit_to_ars(query)

                    sleep(400)

                    result_status = retrieve_ars_results(kcresult, name, check_sheet)


                    dict_workflows[filename] = kcresult

                    sleep(100)

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.0_RHOBTB2_direct.json
A.0_RHOBTB2_direct
https://arax.ncats.io/?source=ARS&id=d9dc26e5-23f3-49c4-8268-47bbc385b695
Done
Done
ara-aragorn Done 0
Done
ara-arax Done 0
Done
ara-bte Done 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.0_RHOBTB2_direct_inverse.json
A.0_RHOBTB2_direct_inverse
https://arax.ncats.io/?source=ARS&id=40a7a4e4-ccf2-4e1d-aabc-01d3c8eb3c83
Done
Done
ara-aragorn Done 3
Done
ara-arax Done 4
Done
ara-bte Done 4
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.2_RHOBTB2_twohop.json
A.2_RHOBTB2_twohop
https://arax.ncats.io/?source=ARS&id=0b43fe59-bb94-4287-b92f-aedb3290fad4
Done
Done
ara-arax Done 500
Done
ara-aragorn Done 8510
Done
ara-bte ARS Error 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.2a_expanded_RHOBTB2_twohop.json
A.2a_expanded_RHOBTB2_twohop
https://arax.ncats.io/?source=ARS&id=ffbd81d1-efbf-4a2d-a148-f0bef293b

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowC/C.1a_SmallMolecule_real_world_evidence_MultSclerosis.json
C.1a_SmallMolecule_real_world_evidence_MultSclerosis
https://arax.ncats.io/?source=ARS&id=b9a0dbf9-ab0b-4b47-9fc4-90a12877096d
Done
Done
C.1a_SmallMolecule_real_world_evidence_MultSclerosis.json
0 MONDO:0005301
curie id: MONDO:0005301 : INCLUDED at postion N == [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99] on n00
ara-aragorn Done 255
Done
C.1a_SmallMolecule_real_world_evidence_MultSclerosis.json
0 MONDO:0005301
curie id: MONDO:0005301 : INCLUDED at postion N == [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 51, 52, 5

<br>

### Codes below are for recording messages and generating outout as csv

<br>

In [8]:
dict_workflows

{'A.0_RHOBTB2_direct': 'd9dc26e5-23f3-49c4-8268-47bbc385b695',
 'A.0_RHOBTB2_direct_inverse': '40a7a4e4-ccf2-4e1d-aabc-01d3c8eb3c83',
 'A.2_RHOBTB2_twohop': '0b43fe59-bb94-4287-b92f-aedb3290fad4',
 'A.2a_expanded_RHOBTB2_twohop': 'ffbd81d1-efbf-4a2d-a148-f0bef293bb66',
 'B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol': '1760155c-f2db-4c08-9f33-0f1e07a88f2b',
 'B.3b_DILI-two-hop-drug': '455d6829-ebe4-4d06-93ab-9b34523009d6',
 'B.1_DILI-three-hop': 'f3558327-499a-4264-bdf2-f2911c10bac6',
 'B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone': 'e4652b33-9e50-410f-a872-a14bdcacefd7',
 'B.3a_DILI-two-hop-chemical-entity': '795d9c3a-d648-4264-bb9b-55c2f103e581',
 'B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone': '62e2663b-f362-47f2-9eab-7a4990833c6c',
 'B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin': '3b1bea37-76e2-4fcd-8518-cd4718f5d9bf',
 'B.2e_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone': '4a640fe0-ea77-4fc2-b94c-022149624796',
 '

In [11]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    name = keys + '.json'
    print(keys, val)
    
    result_status = retrieve_ars_results(val, name, check_sheet)
    
    workflow_result_messages[keys] = result_status

A.0_RHOBTB2_direct d9dc26e5-23f3-49c4-8268-47bbc385b695
Done
Done
ara-aragorn Done 0
Done
ara-arax Done 0
Done
ara-bte Done 0
A.0_RHOBTB2_direct_inverse 40a7a4e4-ccf2-4e1d-aabc-01d3c8eb3c83
Done
Done
ara-aragorn Done 3
Done
ara-arax Done 4
Done
ara-bte Done 4
A.2_RHOBTB2_twohop 0b43fe59-bb94-4287-b92f-aedb3290fad4
Done
Done
ara-arax Done 500
Done
ara-aragorn Done 8510
Done
ara-bte ARS Error 0
A.2a_expanded_RHOBTB2_twohop ffbd81d1-efbf-4a2d-a148-f0bef293bb66
Done
Done
ara-aragorn Done 166
Done
ara-arax Done 35
Done
ara-bte Done 37
B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol 1760155c-f2db-4c08-9f33-0f1e07a88f2b
Done
Done
ara-aragorn Done 731
Done
ara-arax Done 195
Done
ara-bte ARS Error 0
B.3b_DILI-two-hop-drug 455d6829-ebe4-4d06-93ab-9b34523009d6
Done
Done
ara-aragorn Done 922
Done
ara-arax Done 500
Done
ara-bte Done 0
B.1_DILI-three-hop f3558327-499a-4264-bdf2-f2911c10bac6
Done
Done
ara-aragorn Done 0
Done
ara-arax Done 500
Done
ara-bte Done 0
B.2a_DILI-fourth-one-hop-fro

In [12]:
workflow_result_messages

{'A.0_RHOBTB2_direct': [{'pk_id': 'https://arax.ncats.io/?source=ARS&id=d9dc26e5-23f3-49c4-8268-47bbc385b695',
   'ara-aragorn': 'No Results: 200',
   'ara-arax': 'No Results: 200',
   'ara-bte': 'No Results: 200'},
  {}],
 'A.0_RHOBTB2_direct_inverse': [{'pk_id': 'https://arax.ncats.io/?source=ARS&id=40a7a4e4-ccf2-4e1d-aabc-01d3c8eb3c83',
   'ara-aragorn': 'Results: 200 {}',
   'ara-arax': 'Results: 200 {}',
   'ara-bte': 'Results: 200 {}'},
  {'ara-aragorn': {'infores:aragorn',
    'infores:aragorn-ranker-ara',
    'infores:automat-covidkop',
    'infores:automat-ctd',
    'infores:ctd'},
   'ara-arax': {'infores:arax',
    'infores:automat-ctd',
    'infores:biothings-multiomics-biggim-drugresponse',
    'infores:ctd',
    'infores:translator-biothings-explorer'},
   'ara-bte': {'infores:automat-covidkop',
    'infores:automat-ctd',
    'infores:automat-robokop',
    'infores:biothings-multiomics-biggim-drugresponse',
    'infores:ctd',
    'infores:translator-biothings-explorer'}}]

### Creating dataframe for workflows with PK

<br>

In [13]:
## Convert mesages to a dataframe
col = []
final_dict = defaultdict(list)

for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k][0].items():
        #print(key, value)
#         if key.startswith('kp-'):
#             key_mod = key.replace('kp-','')
#         else:
#             key_mod = key
        
        final_dict[key].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

A.0_RHOBTB2_direct
A.0_RHOBTB2_direct_inverse
A.2_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
B.1_DILI-three-hop
B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone
B.2e_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone
B.3a_DILI-two-hop-chemical-entity
B.3b_DILI-two-hop-drug
C.1a_SmallMolecule_real_world_evidence_MultSclerosis
C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule
C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule
C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule
C.3a_MultSclerosis_related_to_Nimodipine
D.1_parkinsons-crohns
D.2_ssri-heart-disease
D.3_ssri-heart-disease-one-hop
D.4_tryptophan-kynurenine
D.6_metformin-ferritin


In [14]:
df

,A.0_RHOBTB2_direct,A.0_RHOBTB2_direct_inverse,A.2_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,B.1_DILI-three-hop,B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.2e_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,...,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
pk_id,https://arax.ncats.io/?source=ARS&id=d9dc26e5-...,https://arax.ncats.io/?source=ARS&id=40a7a4e4-...,https://arax.ncats.io/?source=ARS&id=0b43fe59-...,https://arax.ncats.io/?source=ARS&id=ffbd81d1-...,https://arax.ncats.io/?source=ARS&id=f3558327-...,https://arax.ncats.io/?source=ARS&id=e4652b33-...,https://arax.ncats.io/?source=ARS&id=1760155c-...,https://arax.ncats.io/?source=ARS&id=3b1bea37-...,https://arax.ncats.io/?source=ARS&id=62e2663b-...,https://arax.ncats.io/?source=ARS&id=4a640fe0-...,...,https://arax.ncats.io/?source=ARS&id=b9a0dbf9-...,https://arax.ncats.io/?source=ARS&id=2456df21-...,https://arax.ncats.io/?source=ARS&id=d3d41520-...,https://arax.ncats.io/?source=ARS&id=e24ca073-...,https://arax.ncats.io/?source=ARS&id=a5c5fab1-...,https://arax.ncats.io/?source=ARS&id=65df617a-...,https://arax.ncats.io/?source=ARS&id=2fc8d97f-...,https://arax.ncats.io/?source=ARS&id=439224bd-...,https://arax.ncats.io/?source=ARS&id=3529378c-...,https://arax.ncats.io/?source=ARS&id=012280d4-...
ara-aragorn,No Results: 200,Results: 200 {},Results: 200 {},Results: 200 {},No Results: 200,Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},...,Results: 200 {'MONDO:0005301': 'True'},Results: 200 {'CHEBI:45783': 'False'},Results: 200 {'CHEBI:4875': 'False'},No Results: 200,Results: 200 {},"Results: 200 {'NCBIGene:120892': 'True', 'NCBI...","Results: 200 {'NCBIGene:3988': 'True', 'NCBIGe...",Results: 200 {},No Results: 200,ARS Error: 200
ara-arax,No Results: 200,Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},Results: 200 {},...,Results: 200 {'MONDO:0005301': 'True'},Results: 200 {'CHEBI:45783': 'False'},Results: 200 {'CHEBI:4875': 'False'},Results: 200 {'CHEMBL.COMPOUND:CHEMBL1201607':...,Results: 200 {},"Results: 200 {'NCBIGene:120892': 'False', 'NCB...","Results: 200 {'NCBIGene:3988': 'False', 'NCBIG...",Results: 200 {},ARS Error: 200,ARS Error: 200
ara-bte,No Results: 200,Results: 200 {},ARS Error: 200,Results: 200 {},No Results: 200,ARS Error: 200,ARS Error: 200,Results: 200 {},ARS Error: 200,ARS Error: 200,...,Results: 200 {'MONDO:0005301': 'True'},ARS Error: 200,No Results: 200,Results: 200 {'CHEMBL.COMPOUND:CHEMBL1201607':...,Results: 200 {},ARS Error: 200,ARS Error: 200,Results: 200 {},No Results: 200,ARS Error: 200


In [15]:
df.replace('ARS Error', 'No Results', regex=True,inplace=True)

In [16]:
df.replace('{}','',regex=True,inplace=True)

### Creating second table with edge attribute source

<br>

In [17]:
workflow_result_messages

{'A.0_RHOBTB2_direct': [{'pk_id': 'https://arax.ncats.io/?source=ARS&id=d9dc26e5-23f3-49c4-8268-47bbc385b695',
   'ara-aragorn': 'No Results: 200',
   'ara-arax': 'No Results: 200',
   'ara-bte': 'No Results: 200'},
  {}],
 'A.0_RHOBTB2_direct_inverse': [{'pk_id': 'https://arax.ncats.io/?source=ARS&id=40a7a4e4-ccf2-4e1d-aabc-01d3c8eb3c83',
   'ara-aragorn': 'Results: 200 {}',
   'ara-arax': 'Results: 200 {}',
   'ara-bte': 'Results: 200 {}'},
  {'ara-aragorn': {'infores:aragorn',
    'infores:aragorn-ranker-ara',
    'infores:automat-covidkop',
    'infores:automat-ctd',
    'infores:ctd'},
   'ara-arax': {'infores:arax',
    'infores:automat-ctd',
    'infores:biothings-multiomics-biggim-drugresponse',
    'infores:ctd',
    'infores:translator-biothings-explorer'},
   'ara-bte': {'infores:automat-covidkop',
    'infores:automat-ctd',
    'infores:automat-robokop',
    'infores:biothings-multiomics-biggim-drugresponse',
    'infores:ctd',
    'infores:translator-biothings-explorer'}}]

In [18]:
final_dict2 = defaultdict(dict)
for k in sorted(workflow_result_messages):
    print(k)
    
    
    count = 0
    

    col.append(k)
    for key, value in workflow_result_messages[k][1].items():
        final_dict2[k][key] = value

A.0_RHOBTB2_direct
A.0_RHOBTB2_direct_inverse
A.2_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
B.1_DILI-three-hop
B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone
B.2e_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone
B.3a_DILI-two-hop-chemical-entity
B.3b_DILI-two-hop-drug
C.1a_SmallMolecule_real_world_evidence_MultSclerosis
C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule
C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule
C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule
C.3a_MultSclerosis_related_to_Nimodipine
D.1_parkinsons-crohns
D.2_ssri-heart-disease
D.3_ssri-heart-disease-one-hop
D.4_tryptophan-kynurenine
D.6_metformin-ferritin


In [19]:
final_dict2.keys()

dict_keys(['A.0_RHOBTB2_direct_inverse', 'A.2_RHOBTB2_twohop', 'A.2a_expanded_RHOBTB2_twohop', 'B.1_DILI-three-hop', 'B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone', 'B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol', 'B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin', 'B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone', 'B.2e_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone', 'B.3a_DILI-two-hop-chemical-entity', 'B.3b_DILI-two-hop-drug', 'C.1a_SmallMolecule_real_world_evidence_MultSclerosis', 'C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule', 'C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule', 'C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule', 'C.3a_MultSclerosis_related_to_Nimodipine', 'D.1_parkinsons-crohns', 'D.2_ssri-heart-disease', 'D.3_ssri-heart-disease-one-hop'])

In [20]:
final_dictassemble = []
for k, vs in final_dict2.items():
    #print(k,vs)
    for kv, v in vs.items():
        for t in v:
            final_dictassemble.append([k,kv,t])

In [21]:
final_dictassemble

[['A.0_RHOBTB2_direct_inverse', 'ara-aragorn', 'infores:automat-covidkop'],
 ['A.0_RHOBTB2_direct_inverse', 'ara-aragorn', 'infores:ctd'],
 ['A.0_RHOBTB2_direct_inverse', 'ara-aragorn', 'infores:aragorn-ranker-ara'],
 ['A.0_RHOBTB2_direct_inverse', 'ara-aragorn', 'infores:aragorn'],
 ['A.0_RHOBTB2_direct_inverse', 'ara-aragorn', 'infores:automat-ctd'],
 ['A.0_RHOBTB2_direct_inverse',
  'ara-arax',
  'infores:translator-biothings-explorer'],
 ['A.0_RHOBTB2_direct_inverse', 'ara-arax', 'infores:ctd'],
 ['A.0_RHOBTB2_direct_inverse', 'ara-arax', 'infores:arax'],
 ['A.0_RHOBTB2_direct_inverse',
  'ara-arax',
  'infores:biothings-multiomics-biggim-drugresponse'],
 ['A.0_RHOBTB2_direct_inverse', 'ara-arax', 'infores:automat-ctd'],
 ['A.0_RHOBTB2_direct_inverse',
  'ara-bte',
  'infores:translator-biothings-explorer'],
 ['A.0_RHOBTB2_direct_inverse', 'ara-bte', 'infores:automat-covidkop'],
 ['A.0_RHOBTB2_direct_inverse', 'ara-bte', 'infores:ctd'],
 ['A.0_RHOBTB2_direct_inverse',
  'ara-bte',


In [22]:
column_names = ['Workflow', 'ARS-KPs', 'Values']
df2 = pd.DataFrame(final_dictassemble, columns=column_names)
df2 = df2.astype(str)

In [23]:
df2

,Workflow,ARS-KPs,Values
0,A.0_RHOBTB2_direct_inverse,ara-aragorn,infores:automat-covidkop
1,A.0_RHOBTB2_direct_inverse,ara-aragorn,infores:ctd
2,A.0_RHOBTB2_direct_inverse,ara-aragorn,infores:aragorn-ranker-ara
3,A.0_RHOBTB2_direct_inverse,ara-aragorn,infores:aragorn
4,A.0_RHOBTB2_direct_inverse,ara-aragorn,infores:automat-ctd
...,...,...,...
512,D.3_ssri-heart-disease-one-hop,ara-bte,infores:automat-drug-central
513,D.3_ssri-heart-disease-one-hop,ara-bte,infores:automat-hetio
514,D.3_ssri-heart-disease-one-hop,ara-bte,infores:automat-ctd
515,D.3_ssri-heart-disease-one-hop,ara-bte,infores:biothings-multiomics-clinical-risk


In [24]:
df2.Values = df2.Values.apply(lambda x: x[2:-2] if ('[' in x) else x)

In [25]:
df2test = df2.groupby(['Workflow','Values'])['ARS-KPs'].agg(list)

In [26]:
df2test = pd.DataFrame(df2test.unstack().T)

In [27]:
df2test

Workflow,A.0_RHOBTB2_direct_inverse,A.2_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,B.1_DILI-three-hop,B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.2e_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,B.3a_DILI-two-hop-chemical-entity,B.3b_DILI-two-hop-drug,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop
Values,,,,,,,,,,,,,,,,,,,
CTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-bte],NaN,[ara-aragorn],[ara-aragorn],NaN
DisGeNET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],[ara-aragorn],NaN
DrugCentral Indication,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-bte],NaN,[ara-aragorn],[ara-aragorn],NaN
JensenLab Knowledge GHR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN
JensenLab Knowledge UniProtKB-KW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:sri-reference-kg,NaN,[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],"[ara-aragorn, ara-bte]",[ara-aragorn],[ara-aragorn],NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,[ara-aragorn],[ara-aragorn],"[ara-aragorn, ara-bte]"
infores:stitch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-arax],NaN,NaN,NaN,NaN,[ara-arax],NaN
infores:text-mining-provider-targeted,NaN,NaN,NaN,NaN,NaN,NaN,[ara-bte],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[ara-aragorn, ara-bte]",NaN,[ara-arax],[ara-bte]


In [28]:
#df2test.drop([''], axis=0, inplace=True)

In [29]:
#df2test.index = df2test.index.map(lambda x: x[2:-2] if ('[' in x) else x)

In [30]:
#df2.replace([], 'None', regex=True,inplace=True)

In [31]:
#df2test = df2test.mask(df2test.applymap(type).eq(list) & ~df2test.astype(bool))

In [32]:
df2test = df2test.rename_axis(None)

In [33]:
df2test.columns.name = None

In [34]:
df2test.fillna('', inplace=True)

In [35]:
df2test

,A.0_RHOBTB2_direct_inverse,A.2_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,B.1_DILI-three-hop,B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.2e_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,B.3a_DILI-two-hop-chemical-entity,B.3b_DILI-two-hop-drug,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop
CTD,,,,,,,,,,,,,,,[ara-bte],,[ara-aragorn],[ara-aragorn],
DisGeNET,,,,,,,,,,,,,,,,,[ara-aragorn],[ara-aragorn],
DrugCentral Indication,,,,,,,,,,,,,,,[ara-bte],,[ara-aragorn],[ara-aragorn],
JensenLab Knowledge GHR,,,,,,,,,,,,,,,,,[ara-aragorn],,
JensenLab Knowledge UniProtKB-KW,,,,,,,,,,,,,,,,,[ara-aragorn],,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:sri-reference-kg,,[ara-aragorn],[ara-aragorn],,[ara-aragorn],[ara-aragorn],"[ara-aragorn, ara-bte]",[ara-aragorn],[ara-aragorn],,,,,[ara-aragorn],,,[ara-aragorn],[ara-aragorn],"[ara-aragorn, ara-bte]"
infores:stitch,,,,,,,,,,,,,[ara-arax],,,,,[ara-arax],
infores:text-mining-provider-targeted,,,,,,,[ara-bte],,,,,,,,,"[ara-aragorn, ara-bte]",,[ara-arax],[ara-bte]
infores:translator-biothings-explorer,"[ara-arax, ara-bte]",[ara-arax],"[ara-arax, ara-bte]",[ara-arax],,,[ara-bte],,,[ara-arax],[ara-arax],"[ara-arax, ara-bte]","[ara-aragorn, ara-arax]",[ara-aragorn],"[ara-arax, ara-bte]","[ara-aragorn, ara-bte]",[ara-arax],[ara-arax],[ara-bte]


In [36]:
infores_catalog = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/InfoRes Catalog - Translator InfoRes Catalog.csv", header=1)

In [37]:
infores_catalog = infores_catalog[['id', 'name','translator category','has contributor']]
infores_catalog = infores_catalog[:335]
dict_map = {}
for i in df2test.index.values:
    if i in infores_catalog['id'].values:
        indices = infores_catalog[infores_catalog['id']==i].index[0]
        if pd.notnull(infores_catalog.iloc[indices]['has contributor']):
            dict_map[i] = infores_catalog.iloc[indices]['translator category']
        else:
            dict_map[i] = 'External Source'
    else:
        dict_map[i] = 'Illegal value'

        
        
        
        
        
        
df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog']=df2test.index.map(dict_map)
df2test['sort']=pd.Categorical(df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog'], ["KP", "ARA",'External Source', 'Illegal Value'])
df2test =df2test.sort_values(['sort'])
df2test  = df2test.rename_axis(None)
cols = df2test.columns.tolist()
cols = [cols[-2]] + cols[:-2]
df2test = df2test[cols]
df2test['Query Type'] = 'Async'

df2test = df2test[['Query Type']+ list(df2test.columns[:-1])]

In [38]:
df2test

,Query Type,Translator_Category_Complaint_to_ColL&M_InforesCatalog,A.0_RHOBTB2_direct_inverse,A.2_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,B.1_DILI-three-hop,B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,...,B.3a_DILI-two-hop-chemical-entity,B.3b_DILI-two-hop-drug,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop
infores:biothings-dgidb,Async,KP,,,,,,,,,...,,,,"[ara-aragorn, ara-arax]",[ara-aragorn],"[ara-arax, ara-bte]",,,[ara-arax],
infores:text-mining-provider-targeted,Async,KP,,,,,,,[ara-bte],,...,,,,,,,"[ara-aragorn, ara-bte]",,[ara-arax],[ara-bte]
infores:lincs,Async,KP,,"[ara-arax, ara-aragorn]",[ara-aragorn],,,,,,...,,,,,,,,[ara-aragorn],[ara-aragorn],
infores:molepro,Async,KP,,[ara-arax],"[ara-aragorn, ara-arax, ara-bte]",,"[ara-aragorn, ara-arax]",[ara-aragorn],"[ara-aragorn, ara-bte]","[ara-aragorn, ara-arax]",...,,,,[ara-arax],"[ara-aragorn, ara-arax]",[ara-bte],,[ara-arax],"[ara-aragorn, ara-arax]","[ara-aragorn, ara-bte]"
infores:mychem-info,Async,KP,,[ara-arax],,,,,,,...,,,,"[ara-aragorn, ara-arax]",,[ara-arax],,,[ara-arax],
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:faers,Async,Illegal value,,[ara-aragorn],,,[ara-aragorn],[ara-aragorn],,[ara-aragorn],...,,,[ara-aragorn],,[ara-aragorn],,,[ara-aragorn],[ara-aragorn],"[ara-aragorn, ara-bte]"
infores:hetio,Async,Illegal value,,"[ara-arax, ara-aragorn]",[ara-aragorn],,[ara-aragorn],[ara-aragorn],,[ara-aragorn],...,,,[ara-aragorn],"[ara-aragorn, ara-arax]",[ara-aragorn],[ara-bte],,[ara-aragorn],"[ara-aragorn, ara-arax]","[ara-aragorn, ara-bte]"
infores:ontological-hierarchy,Async,Illegal value,,[ara-aragorn],[ara-aragorn],,[ara-aragorn],[ara-aragorn],,[ara-aragorn],...,,[ara-aragorn],[ara-aragorn],,,,,[ara-aragorn],[ara-aragorn],[ara-aragorn]
infores:pdsp,Async,Illegal value,,,,,,,,,...,,,,,,,,,[ara-arax],


<br>

## Converting the Pk's to hyperlink

<br>

In [39]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [40]:
df.rename({'pk_id': 'pk'}, inplace=True)

In [41]:
df.sort_index(inplace=True)

In [42]:
df['Query Type'] = 'Async'

df = df[['Query Type']+ list(df.columns[:-1])]

In [43]:
df

,Query Type,A.0_RHOBTB2_direct,A.0_RHOBTB2_direct_inverse,A.2_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,B.1_DILI-three-hop,B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,...,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,Async,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,...,Results: 200 {'MONDO:0005301': 'True'},Results: 200 {'CHEBI:45783': 'False'},Results: 200 {'CHEBI:4875': 'False'},No Results: 200,Results: 200,"Results: 200 {'NCBIGene:120892': 'True', 'NCBI...","Results: 200 {'NCBIGene:3988': 'True', 'NCBIGe...",Results: 200,No Results: 200,No Results: 200
ara-arax,Async,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,...,Results: 200 {'MONDO:0005301': 'True'},Results: 200 {'CHEBI:45783': 'False'},Results: 200 {'CHEBI:4875': 'False'},Results: 200 {'CHEMBL.COMPOUND:CHEMBL1201607':...,Results: 200,"Results: 200 {'NCBIGene:120892': 'False', 'NCB...","Results: 200 {'NCBIGene:3988': 'False', 'NCBIG...",Results: 200,No Results: 200,No Results: 200
ara-bte,Async,No Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,...,Results: 200 {'MONDO:0005301': 'True'},No Results: 200,No Results: 200,Results: 200 {'CHEMBL.COMPOUND:CHEMBL1201607':...,Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200
pk,Async,"=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...",...,"=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&..."


In [44]:
#date = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
#wks_name = 'Workflow Progress Tracker Asynchronous_' + date

In [45]:
#wks_name

<br>

### Pushing dataframe to excel sheet on google drive

<br>


**Here I am using the google drive API to push the daatframe into an axcel sheet 
Every individula has the unique credential file that they need to create for google drive API -- 
"araxworkflowprogresstesting-2632632db8be.json" -- is the credential used from my drive. place this json file where
the ReadAndRunAllWorkFLows.ipynb will be. NB: i have removed my credntial file for privacy reasons. Always remove
the json file before making committs to the repo. To use googe Drive API follow: https://towardsdatascience.com/how-to-manage-files-in-google-drive-with-python-d26471d91ecd**

<br>

<br>

## Google Drive API

**Push the dataframe to a google sheet via google drive API and then format the google spread sheet to add hyperlink to pk's and color the cells

<br>

### Push Dataframe 1

<br>

In [46]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
from gspread_formatting import *
import gspread_dataframe as gd

In [47]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'araxworkflowprogresstesting-2632632db8be.json', scope)
gc = gspread.authorize(credentials)

In [48]:
ws = gc.open("workflow_progress_tracker").worksheet("Workflow Progress Tracker_2021_09_06-08_06_07_AM")

#existing = gd.get_as_dataframe(ws)
existing = pd.DataFrame(ws.get_all_records())

existing.set_index('', inplace=True)



existing = existing.rename_axis(None)
existing.loc['pk'][:] = existing.loc['pk'][:].apply(lambda x: x.replace('" "', '","'))
existing.loc['pk'][1]

updated = existing.append(df)
gd.set_with_dataframe(ws, updated,include_index=True,include_column_header=True)

In [ ]:
#spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
#d2g.upload(df, spreadsheet_key, "TESTING ONLY", credentials=credentials, row_names=True)

In [49]:
gc = gspread.service_account(filename='/Users/priyash/Documents/GitHub/minihackathons/Notebooks/araxworkflowprogresstesting-2632632db8be.json')
wksh = gc.open("workflow_progress_tracker")
#sh = wksh.worksheet(wks_name)
sh = wksh.worksheet("Workflow Progress Tracker_2021_09_06-08_06_07_AM")

In [51]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(1,0,0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{}]}

In [52]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.5, 0.0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 565196289,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 1}}}}}}},
  {},
  {}]}

In [53]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['No Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0.75, 0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 565196289,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Results'}]},
      'format': {'backgroundColor': {'green': 0.49803922},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.49803922}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 565196289,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red

In [54]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_CONTAINS', ['Error: 501']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.75, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 565196289,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'No Results'}]},
      'format': {'backgroundColor': {'red': 0.7490196, 'green': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 0.7490196,
         'green': 0.7490196}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 565196289,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Results'}]},
      'format': {'backgroundColor': {'green': 0.49803922},
       'textFormat': {'bold

In [55]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Unknown']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 565196289,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_CONTAINS',
       'values': [{'userEnteredValue': 'Error: 501'}]},
      'format': {'backgroundColor': {'green': 0.7490196, 'blue': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.7490196,
         'blue': 0.7490196}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 565196289,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'No Results'}]},
      'format': {'backgroundColor': {'red': 0.7490196, 'green': 0.7490196},
       '

In [ ]:
# Select a range
cell_list = sh.range('B19:AS19')

# Update in batch
sh.update_cells(cell_list,value_input_option='USER_ENTERED')


In [ ]:
# Select a range
cell_list = sh.range('B23:AS23')

# Update in batch
sh.update_cells(cell_list,value_input_option='USER_ENTERED')


In [58]:
set_frozen(sh, cols=1, rows=1)

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{}]}

### Push Dataframe 2

<br>

In [59]:
df2test

,Query Type,Translator_Category_Complaint_to_ColL&M_InforesCatalog,A.0_RHOBTB2_direct_inverse,A.2_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,B.1_DILI-three-hop,B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.2d_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,...,B.3a_DILI-two-hop-chemical-entity,B.3b_DILI-two-hop-drug,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop
infores:biothings-dgidb,Async,KP,,,,,,,,,...,,,,"[ara-aragorn, ara-arax]",[ara-aragorn],"[ara-arax, ara-bte]",,,[ara-arax],
infores:text-mining-provider-targeted,Async,KP,,,,,,,[ara-bte],,...,,,,,,,"[ara-aragorn, ara-bte]",,[ara-arax],[ara-bte]
infores:lincs,Async,KP,,"[ara-arax, ara-aragorn]",[ara-aragorn],,,,,,...,,,,,,,,[ara-aragorn],[ara-aragorn],
infores:molepro,Async,KP,,[ara-arax],"[ara-aragorn, ara-arax, ara-bte]",,"[ara-aragorn, ara-arax]",[ara-aragorn],"[ara-aragorn, ara-bte]","[ara-aragorn, ara-arax]",...,,,,[ara-arax],"[ara-aragorn, ara-arax]",[ara-bte],,[ara-arax],"[ara-aragorn, ara-arax]","[ara-aragorn, ara-bte]"
infores:mychem-info,Async,KP,,[ara-arax],,,,,,,...,,,,"[ara-aragorn, ara-arax]",,[ara-arax],,,[ara-arax],
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:faers,Async,Illegal value,,[ara-aragorn],,,[ara-aragorn],[ara-aragorn],,[ara-aragorn],...,,,[ara-aragorn],,[ara-aragorn],,,[ara-aragorn],[ara-aragorn],"[ara-aragorn, ara-bte]"
infores:hetio,Async,Illegal value,,"[ara-arax, ara-aragorn]",[ara-aragorn],,[ara-aragorn],[ara-aragorn],,[ara-aragorn],...,,,[ara-aragorn],"[ara-aragorn, ara-arax]",[ara-aragorn],[ara-bte],,[ara-aragorn],"[ara-aragorn, ara-arax]","[ara-aragorn, ara-bte]"
infores:ontological-hierarchy,Async,Illegal value,,[ara-aragorn],[ara-aragorn],,[ara-aragorn],[ara-aragorn],,[ara-aragorn],...,,[ara-aragorn],[ara-aragorn],,,,,[ara-aragorn],[ara-aragorn],[ara-aragorn]
infores:pdsp,Async,Illegal value,,,,,,,,,...,,,,,,,,,[ara-arax],


In [60]:
ws2 = gc.open("workflow_progress_tracker").worksheet("edge_attribute_source_2021_09_06-08_06_07_AM")

#existing = gd.get_as_dataframe(ws)
existing = pd.DataFrame(ws2.get_all_records())

existing.set_index('', inplace=True)



existing = existing.rename_axis(None)

updated = existing.append(df2test)
gd.set_with_dataframe(ws2, updated,include_index=True,include_column_header=True)


In [61]:
set_frozen(ws2, cols=1, rows =1)

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{}]}